In [63]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

# Example dataframe
df = pd.read_csv("Iowa_Liquor_Sales_20250926.csv", usecols=["Store Number", "Store Location"])
df = df.drop_duplicates(subset=["Store Number"], keep="first")
df = df.dropna()

In [61]:
df2 = pd.read_csv("Iowa_Liquor_Sales_20250926.csv")
df2

,Date,Store Number,City,Zip Code,Store Location,County Number,County,Category,Vendor Number,Item Number,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,01/02/2023,5245,DES MOINES,50315.0,POINT (-93.61379 41.57608),NaN,POLK,1022200,260,89177,6,750,29.99,44.99,3,134.97,2.25,0.59
1,01/02/2023,6303,DES MOINES,50320.0,POINT (-93.59738 41.55503),NaN,POLK,1022100,395,89193,48,200,3.63,5.45,48,261.60,9.60,2.53
2,01/02/2023,6303,DES MOINES,50320.0,POINT (-93.59738 41.55503),NaN,POLK,1012200,260,4626,12,750,20.99,31.49,12,377.88,9.00,2.37
3,01/02/2023,2190,DES MOINES,50314.0,POINT (-93.61979 41.60558),NaN,POLK,1032100,978,34598,6,1750,14.84,22.26,6,130.50,10.50,2.77
4,01/02/2023,2699,ANKENY,50023.0,POINT (-93.62362 41.70324),NaN,POLK,1031100,301,38176,12,750,10.00,15.00,24,360.00,18.00,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858345,08/29/2025,4823,CRESTON,50801.0,POINT (-94.37257 41.0668),NaN,UNION,1012100,260,10808,12,1000,20.49,30.74,6,184.44,6.00,1.58
6858346,08/29/2025,4346,SHELLSBURG,52332.0,POINT (-91.86889 42.09427),NaN,BENTON,1081400,434,81206,12,750,4.25,6.38,3,19.14,2.25,0.59
6858347,08/29/2025,4596,DES MOINES,50317.0,POINT (-93.55924 41.6003),NaN,POLK,1031100,421,35314,24,375,1.71,2.57,4,10.28,1.50,0.39
6858348,08/29/2025,2524,DUBUQUE,52001.0,POINT (-90.72265 42.49105),NaN,DUBUQUE,1082100,35,66836,6,750,20.00,30.00,4,120.00,3.00,0.79


In [20]:
from tqdm import tqdm

In [66]:
from math import radians

df[["lon", "lat"]] = (
    df["Store Location"]
    .str.replace("POINT \(", "", regex=True)
    .str.replace("\)", "", regex=True)
    .str.split(" ", expand=True)
    .astype(float)
)
df["lat"] = df['lat'].astype(float).apply(radians)
df["lon"] = df['lon'].astype(float).apply(radians)

In [64]:
df

,Store Number,Store Location
0,5245,POINT (-93.61379 41.57608)
1,6303,POINT (-93.59738 41.55503)
3,2190,POINT (-93.61979 41.60558)
4,2699,POINT (-93.62362 41.70324)
5,2636,POINT (-93.57411 41.60444)
...,...,...
6835250,10653,POINT (-95.12921 43.38217)
6847204,10639,POINT (-91.4257 41.48638)
6847490,10652,POINT (-93.71378 41.77213)
6853047,10657,POINT (-91.57717 42.03488)


In [67]:
from sklearn.metrics.pairwise import haversine_distances
result = haversine_distances(df[["lat", "lon"]])






In [68]:
pairwise_distance_miles = result * 3959

In [69]:
np.fill_diagonal(pairwise_distance_miles, np.NaN)
pairwise_distance_miles

array([[         nan,   1.6838412 ,   2.06182724, ...,  14.49627859,
        109.58102776, 107.17792933],
       [  1.6838412 ,          nan,   3.6799311 , ...,  16.15968611,
        109.22127709, 106.82643472],
       [  2.06182724,   3.6799311 ,          nan, ...,  12.48839053,
        109.28296777, 106.87041282],
       ...,
       [ 14.49627859,  16.15968611,  12.48839053, ...,          nan,
        111.36677244, 108.91559646],
       [109.58102776, 109.22127709, 109.28296777, ..., 111.36677244,
                 nan,   2.45965014],
       [107.17792933, 106.82643472, 106.87041282, ..., 108.91559646,
          2.45965014,          nan]])

In [70]:
df["# of stores within 5 mile radius"] = (pairwise_distance_miles < 5).sum(axis=1)


In [71]:
df["Nearest other store (mi)"] = pd.DataFrame(pairwise_distance_miles).min(axis=1, skipna=True).to_numpy()


In [72]:
df

,Store Number,Store Location,lon,lat,# of stores within 5 mile radius,Nearest other store (mi)
0,5245,POINT (-93.61379 41.57608),-1.633869,0.725639,86,0.566167
1,6303,POINT (-93.59738 41.55503),-1.633582,0.725272,64,0.051137
3,2190,POINT (-93.61979 41.60558),-1.633974,0.726154,98,0.024184
4,2699,POINT (-93.62362 41.70324),-1.634040,0.727859,63,0.140796
5,2636,POINT (-93.57411 41.60444),-1.633176,0.726134,80,0.298991
...,...,...,...,...,...,...
6835250,10653,POINT (-95.12921 43.38217),-1.660318,0.757162,21,0.007031
6847204,10639,POINT (-91.4257 41.48638),-1.595679,0.724074,1,0.258425
6847490,10652,POINT (-93.71378 41.77213),-1.635614,0.729061,6,0.018401
6853047,10657,POINT (-91.57717 42.03488),-1.598323,0.733647,66,0.020202


In [73]:
df.to_csv("proximity.csv")